# Create Data Tables For HiFi Data QC<a class="tocSkip">

**This notebook automatically reads in data stored in the AnVIL_HPRC workspace's bucket and generates data table so the data can be run through QC (NTSM and ReadStat's WDLs)**

**Jeltje changed this in Feb 2023 to output per-file tables instead of per-sample**

**Below are the steps taken in this notebook:**
1. Import Statements & Global Variable Definitions
2. Define Functions
3. Read In Sample Names
4. Create Dataframe Of Files
5. Write data frame to data tables

# Import Statements & Global Variable Definitions

## Installs

In [1]:
%%capture
%pip install gcsfs
## capture CANNOT have comments above it
## For reading CSVs stored in Google Cloud (without downloading them first)
## May need to restart kernel after install 

In [2]:
%%capture
%pip install --upgrade --no-cache-dir --force-reinstall terra-pandas
%pip install --upgrade --no-cache-dir  --force-reinstall git+https://github.com/DataBiosphere/terra-notebook-utils
## For reading/writing data tables into pandas data frames
## May need to restart kernel after install 

## Import Statements

In [3]:
from firecloud import fiss
import pandas as pd         
import os                 
import subprocess       
import re                 
import io
import gcsfs

from typing import Any, Callable, List, Optional
from terra_notebook_utils import table, WORKSPACE_NAME, WORKSPACE_GOOGLE_PROJECT
from terra_pandas import dataframe_to_table, table_to_dataframe

## Global Variable Declarations

In [4]:
# AnVIL_HPRC WorkspaceBucket
anvil_hprc_bucket       = "gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/"

# Get the Google billing project name and workspace name for current workspace
PROJECT = os.environ['WORKSPACE_NAMESPACE']
WORKSPACE =os.path.basename(os.path.dirname(os.getcwd()))
bucket = os.environ['WORKSPACE_BUCKET'] + "/"


# Verify that we've captured the environment variables
print("Billing project: " + PROJECT)
print("Workspace: " + WORKSPACE)
print("Workspace storage bucket: " + bucket)

Billing project: human-pangenome-ucsc
Workspace: HPRC_WRANGLING_WUSTL_HPRC_HiFi_Year3
Workspace storage bucket: gs://fc-a7e6ae6b-860b-4519-80a5-277aeb967124/


# Function Definitions

In [5]:
def rtn_datatype_ls_for_subm(bucket_url, submission_key, data_type_subdir, file_type_ls):
    '''Takes in:
            * bucket_url (string): url of bucket to search (should be the AnVIL_HPRC bucket)
                ex: "gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/"
            * submission_key (string): the UUID plus the submission name to search
                ex: "5c68b972-8534-402f-9861-11c93558765f--UW_HPRC_HiFi_Y3"
            * data_type_subdir (string): name of the subfolder to search 
              (used when a submission has more than one data type.)
                ex: "PacBio_HiFi" if the data is not in subfolders, pass "."
            * file_type_ls (list of strings): file extensions to search for. Often a submission will 
              have more than one type of file that represents the same dataset. Be careful to not 
              include replicate data, however.
                ex: ".hifi_reads.bam"
     then performs a list of the bucket, then returns a filtered list files.'''
    
    rtn_file_ls = []
    
    submission_path = str(bucket_url + "submissions/" + submission_key)
    file_list_byt   = subprocess.run(['gsutil', '-u', 'human-pangenome-ucsc', 'ls', '-r', submission_path], 
                                     stdout=subprocess.PIPE)

    file_list_str   = file_list_byt.stdout.decode('utf-8')
    file_list       = file_list_str.split('\n')  ## Pull out "\n"
   
    ## filter out empty strings
    file_list = [ elem for elem in file_list if elem != '']
    
    ## Pull from subdir type we are targeting
    file_list = list(filter(lambda x:re.search(rf"{data_type_subdir}", x), file_list))
    
    for file_type in file_type_ls:
    
        ## Pull files of correct type (ex: ccs.bam)
        file_list_by_type = list(filter(lambda x:re.search(rf"{file_type}$", x), file_list))

        ## Add to list of files to return
        rtn_file_ls += file_list_by_type

    return rtn_file_ls    

# Read In Sample Names

In [6]:
## This file (a list of sample IDs) should be stored in the Terra workspace (in the bucket) that is being used for 
## the submission you  are wrangling...
sample_info_fp = "gs://fc-a7e6ae6b-860b-4519-80a5-277aeb967124/WUSTL_HPRC_HiFi_Year3.csv"

sample_df = pd.read_csv(sample_info_fp, header=None)

sample_df.rename(columns = {0:'sample_id'}, inplace = True)

# Create Dataframe Of Files

## Find the data

In [7]:
subm_key         = "c0de0f97-f422-4057-90bd-12b40869d30a--WUSTL_HPRC_HiFi_Year3"
data_type_subdir = "."
file_type_ls     = [".hifi_reads.bam"]

In [8]:
## get a list of the files in submission (subm_key) that match the 
## data_type_subdir and file_type_ls
## NOTE: if this gets a service account warning, wait 24 hours (?) for the workspace to get added
file_ls  = rtn_datatype_ls_for_subm(anvil_hprc_bucket, subm_key, 
                                      data_type_subdir, file_type_ls)

In [9]:
## Check that the number of files matches what we expect
len(file_ls)

79

## Add Sample Data To File Data Frame

In [10]:
filedict = dict()

for f in file_ls:
    for s in sample_df.sample_id:
        if re.search(s,f):
            filedict[f] = s
            break            
print(len(filedict))
file_df = pd.DataFrame.from_dict(filedict.items())
file_df.columns = ['hifi', 'sample']


79


## Add 1000G data (for NTSM run)

In [11]:
## Read in 1000G data from another workspace
## We will be using this to compare against out submission to make sure that
## the data comes from the same samples
thousand_genomes_df = table_to_dataframe("sample", 
                                        workspace="1000G-high-coverage-2019", 
                                        workspace_namespace="anvil-datastorage")
## Use the 1kg library name (i.e. HG00621) as the index (that matches our sample name)
thousand_genomes_df.set_index('library_name', inplace=True)

## We only need the cram file (represents 30X Ilmn dataset)
thousand_genomes_df = thousand_genomes_df[['cram']]

## name the column to be a bit more descriptive
thousand_genomes_df.rename(columns = {'cram':'1000g_cram'}, inplace = True)

In [12]:
## merge the two dataframes
ntsm_df1 = pd.merge(
    file_df,
    thousand_genomes_df,
    left_on='sample',
    right_index=True)
ntsm_df1.head()

,hifi,sample,1000g_cram
0,gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/s...,HG00140,gs://fc-56ac46ea-efc4-4683-b6d5-6d95bed41c5e/C...
1,gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/s...,HG00140,gs://fc-56ac46ea-efc4-4683-b6d5-6d95bed41c5e/C...
2,gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/s...,HG00140,gs://fc-56ac46ea-efc4-4683-b6d5-6d95bed41c5e/C...
3,gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/s...,HG00140,gs://fc-56ac46ea-efc4-4683-b6d5-6d95bed41c5e/C...
4,gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/s...,HG00323,gs://fc-56ac46ea-efc4-4683-b6d5-6d95bed41c5e/C...


In [21]:
set(file_df['sample']) - set(thousand_genomes_df.index.tolist())
#file_df['sample']
#ntsm_df.index.tolist()

{'MGISTL_PAN027_HG06807'}

In [13]:
## Read in 1000G data from another workspace
## instead of cram files (as before) we'll use fastq files directly so we're not dependent
## on the public hg18.fa file, which has been giving problems.
## We will be using this to compare against out submission to make sure that
## the data comes from the same samples
t2t_reads_df = table_to_dataframe("participant", 
                                 workspace_namespace="anvil-datastorage", 
                                 workspace="AnVIL_T2T")
t2t_reads_df = t2t_reads_df[['read_1_fastq', 'read_2_fastq']]

t2t_reads_df.head()

,read_1_fastq,read_2_fastq
participant_id,,
HG00096,gs://fc-47de7dae-e8e6-429c-b760-b4ba49136eee/1...,gs://fc-47de7dae-e8e6-429c-b760-b4ba49136eee/1...
HG00097,gs://fc-47de7dae-e8e6-429c-b760-b4ba49136eee/1...,gs://fc-47de7dae-e8e6-429c-b760-b4ba49136eee/1...
HG00099,gs://fc-47de7dae-e8e6-429c-b760-b4ba49136eee/1...,gs://fc-47de7dae-e8e6-429c-b760-b4ba49136eee/1...
HG00100,gs://fc-47de7dae-e8e6-429c-b760-b4ba49136eee/1...,gs://fc-47de7dae-e8e6-429c-b760-b4ba49136eee/1...
HG00101,gs://fc-47de7dae-e8e6-429c-b760-b4ba49136eee/1...,gs://fc-47de7dae-e8e6-429c-b760-b4ba49136eee/1...


In [22]:
## merge the two dataframes
ntsm_df = pd.merge(
    ntsm_df1,
    t2t_reads_df,
    left_on='sample',
    right_index=True)

In [24]:
print(ntsm_df.shape)
ntsm_df.head()

(75, 5)


,hifi,sample,1000g_cram,read_1_fastq,read_2_fastq
0,gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/s...,HG00140,gs://fc-56ac46ea-efc4-4683-b6d5-6d95bed41c5e/C...,gs://fc-47de7dae-e8e6-429c-b760-b4ba49136eee/1...,gs://fc-47de7dae-e8e6-429c-b760-b4ba49136eee/1...
1,gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/s...,HG00140,gs://fc-56ac46ea-efc4-4683-b6d5-6d95bed41c5e/C...,gs://fc-47de7dae-e8e6-429c-b760-b4ba49136eee/1...,gs://fc-47de7dae-e8e6-429c-b760-b4ba49136eee/1...
2,gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/s...,HG00140,gs://fc-56ac46ea-efc4-4683-b6d5-6d95bed41c5e/C...,gs://fc-47de7dae-e8e6-429c-b760-b4ba49136eee/1...,gs://fc-47de7dae-e8e6-429c-b760-b4ba49136eee/1...
3,gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/s...,HG00140,gs://fc-56ac46ea-efc4-4683-b6d5-6d95bed41c5e/C...,gs://fc-47de7dae-e8e6-429c-b760-b4ba49136eee/1...,gs://fc-47de7dae-e8e6-429c-b760-b4ba49136eee/1...
4,gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/s...,HG00323,gs://fc-56ac46ea-efc4-4683-b6d5-6d95bed41c5e/C...,gs://fc-47de7dae-e8e6-429c-b760-b4ba49136eee/1...,gs://fc-47de7dae-e8e6-429c-b760-b4ba49136eee/1...


# Upload To Tables

In [25]:
## Create tables for running NTSM and ReadStats
dataframe_to_table("ntsm_tmp",      ntsm_df, WORKSPACE, PROJECT)
#dataframe_to_table("readstats", file_df, WORKSPACE, PROJECT)